
# 📘 EDA — `ml_ozon_counterfeit_train.csv`

Этот ноутбук — шаблон для первичного анализа тренировочного датасета хакатона.
Пожалуйста, укажите путь к CSV и (при наличии) к каталогу с изображениями в секции **Config** ниже.

**Что делает ноутбук:**
- Загружает CSV надёжно (поиск файла по маске, обработка русской/латинской `c/с` в имени).
- Показывает размеры, типы колонок, первые строки.
- Проверяет наличие и распределение целевой метки `resolution`.
- Проверяет уникальность `id`.
- Отчёт по пропускам.
- Выделяет вероятные текстовые, категориальные и числовые признаки.
- Базовый анализ текстовых полей (длины строк, пример значений).
- Базовый анализ числовых признаков и их связь с таргетом.
- Поиск колонок, указывающих на изображения (например, `image`, `img_path`, `picture`).

> Примечание: графики строятся через Matplotlib (без seaborn).


In [ ]:

# === Config ===
from pathlib import Path
from glob import glob

# Укажи путь к каталогу с данными (где лежат CSV)
DATA_DIR = Path('.')  # например, Path('/kaggle/input/dataset') или Path('data/raw')

# Явное имя файла (если знаешь точное). Иначе оставь None, и сработает поиск по маске.
TRAIN_CSV_NAME = None  # например, 'ml_ozon_counterfeit_train.csv'

# При наличии картинок можно указать папки (по желанию, используется только для проверки путей)
IMAGES_TRAIN_DIR = None  # например, Path('data/images_train')

# Маски для поиска train-файла, учитывая возможную кириллицу/латиницу в слове "counterfeit"
TRAIN_GLOBS = [
    'ml_ozon_*train*.csv',
    'ml_ozon*train*.csv',
]

def resolve_train_csv():
    if TRAIN_CSV_NAME is not None:
        p = (DATA_DIR / TRAIN_CSV_NAME)
        assert p.exists(), f'Не найден файл {p}'
        return p
    # Поиск по маскам
    candidates = []
    for pat in TRAIN_GLOBS:
        candidates.extend(glob(str(DATA_DIR / pat)))
    # Удалим дубликаты и отсортируем по длине (короче — вероятнее нужный)
    candidates = sorted(set(candidates), key=len)
    assert candidates, f'Не удалось найти train CSV по маскам {TRAIN_GLOBS} в {DATA_DIR.resolve()}'
    print('Кандидаты train CSV:', *candidates, sep='\n- ')
    return Path(candidates[0])

TRAIN_CSV_PATH = resolve_train_csv()
TRAIN_CSV_PATH


In [ ]:

# === Imports ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 200)


In [ ]:

# === Load CSV ===
# Пытаемся читать в UTF-8; если не получится — попробуем cp1251.
read_kwargs = dict(low_memory=False)
try:
    train = pd.read_csv(TRAIN_CSV_PATH, **read_kwargs)
except UnicodeDecodeError:
    train = pd.read_csv(TRAIN_CSV_PATH, encoding='cp1251', **read_kwargs)

print('Shape:', train.shape)
train.head(3)


In [ ]:

# === Info & dtypes ===
print(train.info())


In [ ]:

# === Target check: 'resolution' ===
if 'resolution' in train.columns:
    y = train['resolution'].astype(int)
    print('Target value counts:\n', y.value_counts(dropna=False))
    print('\nClass ratio (positive rate):', y.mean().round(4))
    # Bar chart
    vc = y.value_counts().sort_index()
    plt.figure()
    vc.plot(kind='bar', title='Target distribution (resolution)')
    plt.xlabel('class')
    plt.ylabel('count')
    plt.show()
else:
    print("⚠️ Внимание: колонка 'resolution' не найдена в train. Проверьте название целевой метки.")


In [ ]:

# === ID uniqueness ===
id_col = None
for cand in ['id', 'item_id', 'product_id']:
    if cand in train.columns:
        id_col = cand
        break

if id_col:
    uniq = train[id_col].nunique()
    total = len(train)
    print(f"ID column: {id_col}. Unique: {uniq} / {total}. Duplicates: {total - uniq}")
    if total - uniq > 0:
        dup = train[train.duplicated(id_col, keep=False)].sort_values(id_col)
        print('Примеры дубликатов по ID:')
        display(dup.head(10))
else:
    print('⚠️ Колонка ID не найдена среди типичных имён (id/item_id/product_id). Проверьте вручную.')


In [ ]:

# === Missing values ===
mv = train.isna().sum().sort_values(ascending=False)
mv = mv[mv > 0]
if len(mv) == 0:
    print('Пропусков не обнаружено.')
else:
    miss = pd.DataFrame({'missing': mv, 'missing_%': (mv / len(train) * 100).round(2)})
    display(miss.head(30))


In [ ]:

# === Detect likely text columns ===
obj_cols = [c for c in train.columns if train[c].dtype == 'object']
text_candidates = []
for c in obj_cols:
    # Оценим среднюю длину строк, долю уникальных значений и кол-во NaN
    s = train[c].astype(str)
    lens = s.str.len()
    avg_len = lens.mean()
    uniq_frac = s.nunique(dropna=True) / max(len(s), 1)
    nan_rate = train[c].isna().mean()
    text_candidates.append((c, avg_len, uniq_frac, nan_rate))

text_df = pd.DataFrame(text_candidates, columns=['column', 'avg_len', 'uniq_frac', 'nan_rate']).sort_values('avg_len', ascending=False)
display(text_df.head(20))

# Выберем вероятные текстовые поля (средняя длина >= 20 символов)
likely_text_cols = text_df[text_df['avg_len'] >= 20]['column'].tolist()
print('Вероятные текстовые колонки:', likely_text_cols)


In [ ]:

# === Text length histograms (первые 2) ===
for c in likely_text_cols[:2]:
    lens = train[c].astype(str).str.len()
    plt.figure()
    lens.hist(bins=50)
    plt.title(f'Text length distribution — {c}')
    plt.xlabel('length (chars)')
    plt.ylabel('count')
    plt.show()
    print(train[c].dropna().head(3).tolist())


In [ ]:

# === Categorical candidates ===
cat_candidates = []
for c in obj_cols:
    nunq = train[c].nunique(dropna=True)
    frac = nunq / max(len(train), 1)
    cat_candidates.append((c, nunq, frac))
cat_df = pd.DataFrame(cat_candidates, columns=['column', 'nunique', 'unique_frac']).sort_values(['nunique','column'])
# Эвристика: низкая кардинальность (<500 и <50% уникальности)
likely_cat = cat_df[(cat_df['nunique'] <= 500) & (cat_df['unique_frac'] <= 0.5)]['column'].tolist()
display(cat_df.head(30))
print('Вероятные категориальные:', likely_cat[:20])


In [ ]:

# === Numeric columns analysis ===
num_cols = [c for c in train.columns if pd.api.types.is_numeric_dtype(train[c]) and c != 'resolution']
if num_cols:
    display(train[num_cols].describe().T)
else:
    print('Числовых признаков не обнаружено (кроме таргета, если он числовой).')


In [ ]:

# === Correlation with target (numeric vs binary target) ===
if 'resolution' in train.columns and len(num_cols) > 0:
    corr_rows = []
    y = train['resolution'].astype(float)
    for c in num_cols:
        s = pd.to_numeric(train[c], errors='coerce')
        mask = ~s.isna() & ~y.isna()
        if mask.sum() >= 50:  # минимум наблюдений
            corr = np.corrcoef(s[mask], y[mask])[0,1]
            corr_rows.append((c, corr))
    corr_df = pd.DataFrame(corr_rows, columns=['column', 'pearson_corr_with_target']).sort_values('pearson_corr_with_target', ascending=False)
    display(corr_df.head(20))
else:
    print('Пропускаем корреляцию: нет таргета или числовых колонок.')


In [ ]:

# === Image path columns detection ===
image_like_cols = [c for c in train.columns if any(k in c.lower() for k in ['image', 'img', 'picture', 'photo', 'pic', 'jpg', 'png', 'path'])]
print('Похожие на пути к картинкам колонки:', image_like_cols)
if image_like_cols:
    for c in image_like_cols[:3]:
        print(f'Колонка {c}: пример значений ->')
        print(train[c].dropna().astype(str).head(5).tolist())


In [ ]:

# === High-cardinality id-like columns ===
id_like = []
for c in train.columns:
    nunq = train[c].nunique(dropna=True)
    if nunq > 0.9 * len(train):
        id_like.append(c)
print('ID-like/high-cardinality columns:', id_like)


In [ ]:

# === Save quick summary JSON ===
summary = {
    'shape': tuple(train.shape),
    'columns': train.columns.tolist(),
    'dtypes': {c: str(train[c].dtype) for c in train.columns},
    'has_resolution': 'resolution' in train.columns,
}
summary_path = Path('dataset_summary.json')
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary, f, ensure_ascii=False, indent=2)
print('Saved summary to', summary_path.resolve())



## ✅ Next steps
1. Уточнить ключевые столбцы: `id`, основной текст (`description`/`name`/`title`), пути к изображениям, категориальные/числовые признаки.

2. Зафиксировать списки колонок по типам (text/cat/num) для последующего пайплайна.

3. Проверить дисбаланс по целевой метке и решить, нужен ли тюнинг порога.

4. Подготовить stratifed сплиты и OOF-валидацию по F1.

5. Перейти к построению базового текстового классификатора (TF-IDF + логрег/LinearSVC) как стартовой точки.

